In [2]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline

# 加载 PDF
loaders_chinese = [
    PyMuPDFLoader("/workspaces/LLMEasyProject/database/knowledge_db/pumpkin_book.pdf") # 南瓜书
    # 大家可以自行加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="bert-base-chinese")

persist_directory = '/workspaces/LLMEasyProject/database/vector_db/chroma'
!rm -rf {persist_directory}

/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name bert-base-chinese. Creating a new one with MEAN pooling.


In [3]:
vectordb = Chroma.from_documents(
    documents=split_docs[:100], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [4]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：100


In [5]:
question="什么是机器学习"

sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [6]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
下学生”。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第 1 章和第 2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；
• 每个公式的解析和推导我们都力 (zhi) 争 (neng) 以本科数学基础的视角
--------------
检索到的第1个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我
--------------
检索到的第2个内容: 
提交你希望补充的公式编号或者勘误信息，我们通常会在 24 小时以内给您回复，超过 24 小时未回复的
话可以微信联系我们（微信号：at-Sm1les）；
配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU
在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第 1 版）
最新版 PDF 获取
--------------


In [7]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [8]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我
--------------
MMR 检索到的第1个内容: 
�本�:2.0.0
发布日期:2023.11
南  ⽠  书
PUMPKIN
B  O  O  K
谢�睿 �州 贾彬彬
--------------
MMR 检索到的第2个内容: 
119
10.6 核化线性降维 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
119
→_→
配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU
←_←
--------------


In [1]:
# 导入检索式问答链
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
# !pip install sentencepiece
# 可以使用 HuggingFacePipeline 本地搭建大语言模型
model_id = 'THUDM/chatglm2-6b-int4' # 采用 int 量化后的模型可以节省硬盘占用以及实时量化所需的运算资源
tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True).half().quantize(4).cuda()
model = model.eval()
pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

llm = HuggingFacePipeline(pipeline=pipe)

/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: pip install --upgrade pip


: 